<a href="https://colab.research.google.com/github/Pestrdan/netology_research/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5/RSML_12_HW1_%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Использовать датасет MovieLens.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
#links.head()

In [ ]:
#movies.head()

In [ ]:
#movies.tail()

In [ ]:
#ratings.head()

In [ ]:
#ratings.info()

In [ ]:
#tags.head()

In [ ]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [ ]:
tag_strings = []
moviesId = []

for movieId, group in movies_with_tags.groupby('movieId'):
    tag_strings.append('|'.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    moviesId.append(movieId)

In [ ]:
moviesid_tags_filtered = pd.DataFrame(
    {
        "movieId": moviesId,
        "tags": tag_strings,
    }
)

In [ ]:
moviesid_tags_filtered.head()

,movieId,tags
0,1,pixar|pixar|fun
1,2,fantasy|magicboardgame|RobinWilliams|game
2,3,moldy|old
3,4,nan
4,5,pregnancy|remake


In [ ]:
movies_with_tags_filtered = movies.join(moviesid_tags_filtered.set_index('movieId'), on='movieId')

In [ ]:
movies_with_tags_filtered.head()

,movieId,title,genres,tags
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy|magicboardgame|RobinWilliams|game
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy|old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,nan
4,5,Father of the Bride Part II (1995),Comedy,pregnancy|remake


In [ ]:
movies_with_tags_filtered['tags'] = movies_with_tags_filtered['tags'].fillna('no_tags')

In [ ]:
#movies_with_tags_filtered['movies_with_tags_filtered']= movies_with_tags_filtered['genres'] + '|' + movies_with_tags_filtered['tags']

In [ ]:
#movies_with_tags_filtered.head()

In [ ]:
movies_with_tags_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
 3   tags     9742 non-null   object
dtypes: int64(1), object(3)
memory usage: 304.6+ KB


# **2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:**

**TF-IDF на тегах и жанрах**

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
movie_genres = movies_with_tags_filtered.genres.apply(change_string).tolist()

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movies_with_tags_filtered.genres.values)

In [ ]:
# чтобы понять, вхождение какого тега стоит за каждой координатой
#count_vect.vocabulary_

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
tfidf_matrix = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_transformer.get_feature_names_out())

In [ ]:
#tfidf_matrix.head()

In [ ]:
#movies_with_tags_filtered['movieId']

In [ ]:
tfidf_matrix['movieId'] = movies_with_tags_filtered['movieId']

In [ ]:
#tfidf_matrix.info()

In [ ]:
#tfidf_matrix.reset_index(inplace=True)

In [ ]:
#tfidf_matrix.head()

In [ ]:
#tfidf_matrix.shape[0]

In [ ]:
movie_tags = movies_with_tags_filtered.tags.apply(change_string).tolist()

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movies_with_tags_filtered.genres.values)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

tfidf_matrix2 = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_transformer.get_feature_names_out())

In [ ]:
#tfidf_matrix2.shape[0]

In [ ]:
tfidf_matrix2['movieId'] = movies_with_tags_filtered['movieId']

**средние оценки (+ median, variance и т. д.) пользователя и фильма**

In [ ]:
user_mean = ratings.groupby('userId').agg(
    mean_user=('rating', np.mean),
    median_user=('rating', np.median))

In [ ]:
user_mean.head()

,mean_user,median_user
userId,,
1,4.366379,5.0
2,3.948276,4.0
3,2.435897,0.5
4,3.555556,4.0
5,3.636364,4.0


In [ ]:
user_mean.reset_index(inplace=True)
user_mean.head()

,userId,mean_user,median_user
0,1,4.366379,5.0
1,2,3.948276,4.0
2,3,2.435897,0.5
3,4,3.555556,4.0
4,5,3.636364,4.0


In [ ]:
movie_mean = ratings.groupby('movieId').agg(
    mean_movie=('rating', np.mean),
    median_movie=('rating', np.median))

In [ ]:
movie_mean.head()

,mean_movie,median_movie
movieId,,
1,3.920930,4.0
2,3.431818,3.5
3,3.259615,3.0
4,2.357143,3.0
5,3.071429,3.0


In [ ]:
movie_mean.reset_index(inplace=True)
movie_mean.head()

,movieId,mean_movie,median_movie
0,1,3.920930,4.0
1,2,3.431818,3.5
2,3,3.259615,3.0
3,4,2.357143,3.0
4,5,3.071429,3.0


In [ ]:
movies_with_mean_pre = ratings.join(user_mean.set_index('userId'), on='userId')

In [ ]:
movies_with_mean = movies_with_mean_pre.join(movie_mean.set_index('movieId'), on='movieId')

In [ ]:
#movies_with_mean.head()

In [ ]:
movies_with_mean = movies_with_mean.drop(['timestamp'], axis=1)

In [ ]:
#movies_with_mean.head()

In [ ]:
df_pre = movies_with_mean.join(tfidf_matrix.set_index('movieId'), on='movieId')

In [ ]:
df = df_pre.join(df_pre, on='movieId', how='left', lsuffix='_left', rsuffix='_right')

In [ ]:
df.dropna(inplace=True)

In [ ]:
#df.tail()

In [ ]:
#df.isna()

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95431 entries, 0 to 100607
Data columns (total 62 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   userId_left         95431 non-null  int64  
 1   movieId_left        95431 non-null  int64  
 2   rating_left         95431 non-null  float64
 3   mean_user_left      95431 non-null  float64
 4   median_user_left    95431 non-null  float64
 5   mean_movie_left     95431 non-null  float64
 6   median_movie_left   95431 non-null  float64
 7   x0_left             95431 non-null  float64
 8   x1_left             95431 non-null  float64
 9   x2_left             95431 non-null  float64
 10  x3_left             95431 non-null  float64
 11  x4_left             95431 non-null  float64
 12  x5_left             95431 non-null  float64
 13  x6_left             95431 non-null  float64
 14  x7_left             95431 non-null  float64
 15  x8_left             95431 non-null  float64
 16  x9_

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['rating_left'], axis=1)

In [ ]:
y = df['rating_left']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = KNeighborsRegressor(n_neighbors=10, n_jobs=-1, metric='euclidean')

In [ ]:
model.fit(X_train, y_train)

KNeighborsRegressor(metric='euclidean', n_jobs=-1, n_neighbors=10)

# **3. Оценить RMSE на тестовой выборке.**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
RMSE_test = mean_squared_error(y_test, model.predict(X_test), squared = False)
print(RMSE_test)

1.01945584686757


# **RMSE на тестовой выборке LinearRegression.**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
model.predict(X_test)

array([3.54237136, 4.17740866, 4.13051191, ..., 2.55722351, 3.19593411,
       2.7637162 ])

In [ ]:
RMSE_test = mean_squared_error(y_test, model.predict(X_test), squared = False)
print(RMSE_test)

0.8174076824654755
